<a href="https://colab.research.google.com/github/heinohen/Textual-Data-Analysis/blob/main/project_heinonen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> TDA course project </h1>

Name: **TODO**

Pair: If you did this project as pair work, name the other student here, leave empty otherwise. If you work in pair, <b>both</b> hand out the same project report in Moodle.


# SETUP

### install libraries

In [73]:
!pip3 -q install transformers datasets torch scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00


### import

In [72]:
import datasets as ds
from pprint import pprint
import transformers
from sklearn.model_selection import train_test_split
import torch


#  Step 1: Load the data with LLM judgements


In [46]:
#work here
!wget http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz

--2025-02-13 17:59:58--  http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3373840 (3.2M) [application/octet-stream]
Saving to: ‘tda25-responses.jsonl.gz.1’

tda25-responses.jso 100%[===================>]   3.22M  2.33MB/s    in 1.4s    

2025-02-13 18:00:00 (2.33 MB/s) - ‘tda25-responses.jsonl.gz.1’ saved [3373840/3373840]



Does the article provide a clear sequence of ordered steps that guide the reader through completing a task step-by-step? (Step-by-step: Yes/No)
Does the article contain well-structured procedural instructions that would be useful for training a language model capable of deeper reasoning? (Training: Yes/No)
Briefly explain your answers in 2-3 sentences, focusing on whether the instructions are clearly ordered and detailed enough to encourage deeper reasoning.
The article text starts here:
{{article}}

In [47]:
!gzip -d tda25-responses.jsonl.gz

gzip: tda25-responses.jsonl already exists; do you wish to overwrite (y or n)? ^C


In [79]:
# Process and load this data as a HuggingFace dataset that you can use for supervised training in step 2.
dataset = ds.load_dataset('json', data_files='tda25-responses.jsonl')


dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'response'],
        num_rows: 2515
    })
})

In [80]:
pprint(dataset['train'][460]['response'])

('To correct the error regarding the Fargate task failing in a private subnet '
 'without direct connectivity to the internet, the solution architect should:\n'
 '\n'
 '**D. Ensure the task is set to DISABLED for the auto-assign public IP '
 'setting when launching the task. Configure a NAT gateway in the public '
 'subnet in the VPC to route requests to the internet.**\n'
 '\n'
 'This approach is correct because the task does not require a public IP while '
 'it resides in a private subnet. By configuring a NAT gateway in the public '
 'subnet, the Fargate task can access the internet for any outbound requests '
 'without requiring direct inbound access from the internet, thus maintaining '
 "security and compliance with the architecture's requirements.")


In [81]:
# look at the data

NO_STEP_BY_STEP = []

for i, d in enumerate(dataset['train']):
  answer = ""
  response = d['response'].split()
  response = [item.lower() for item in response]
  response = [item.replace("**", "") for item in response]
  response = [item.replace("(", "") for item in response]
  response = [item.replace(")", "") for item in response]


  try:
    index = response.index("step-by-step:")
  except:
    NO_STEP_BY_STEP.append(i)

In [82]:
NO_STEP_BY_STEP

[460]

In [83]:
indices = [i for i,x in enumerate(dataset['train']) if i not in NO_STEP_BY_STEP]

In [84]:
len(indices)

2514

In [85]:
subdataset = dataset['train'].select(indices)

In [86]:
dataset = ds.DatasetDict({
   'train': subdataset
})

In [87]:

count_of_documents = 0

answers = []

for i, d in enumerate(dataset['train']):
  count_of_documents += 1
  answer = ""
  response = d['response'].split()
  response = [item.lower() for item in response]
  response = [item.replace("**", "") for item in response]
  response = [item.replace("(", "") for item in response]
  response = [item.replace(")", "") for item in response]


  try:
    index = response.index("step-by-step:")
  except:
    print(i)

  assert response[index+1].lower() in ["yes", "no"], f"String does not match 'yes' or 'no'. Found: {response[index+1]}"
  answers.append(response[index+1].lower())

In [88]:
assert len(answers) == len(dataset['train'])
print(count_of_documents)
unique_values = set(answers)
print(unique_values)

2514
{'yes', 'no'}


In [89]:
label_tags = [1 if item == "yes" else 0 for item in answers]

In [90]:
subdataset = subdataset.add_column("label_name", answers)

In [91]:
for i,t in enumerate(label_tags[:10]):
  print(label_tags[i], subdataset[i]['label_name'])

1 yes
0 no
0 no
0 no
0 no
0 no
1 yes
0 no
0 no
1 yes


seems to match...

In [92]:
subdataset = subdataset.add_column("label", label_tags)

In [93]:
dataset = ds.DatasetDict({
   'train': subdataset
})

In [100]:
dataset = dataset.shuffle() # although it is shuffled at train_test_split by default
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'response', 'label_name', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1608
    })
    test: Dataset({
        features: ['document', 'response', 'label_name', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 403
    })
})

huh.... i think its now solved, one example did not have a step-by-step label

In [104]:
# see what we got

print(dataset['train'][0])

{'document': 'Reds Get Madson\nI never would have predicted that in November.\nBack in November it appeared the Phillies and Madson were close to a four-year, $44 million extension. But the negotiations hit a snag (one side said they had a verbal agreement, the other side said they didn’t) and the Phillies decided to spend $6 million more to sign Jonathan Papelbon. Meanwhile, the closing opportunities for Madson dried up and instead of getting the monster pay day he hoped, he has to pitch in Cincinnati for one season before hitting the market again. That’s not exactly how Scott Boras planned it, but that’s the situation.\nIt will be a strange spring training without Madson and possibly Brad Lidge in the clubhouse. They were the final two pieces of the 2008 World Series bullpen. But there is still a chance Lidge is back. He remains unsigned, and the longer he is out there I think the chances improve the Phillies could sign him to a Minor League deal. Read Paul Hagen‘s story on Lidge her

# Step 2: Classifier training and evaluation

Next you should use the data from step (1) to train an encoder-based classifier of your liking and evaluate it properly (i.e. split a test set out of the data and report numerical evaluation metrics assuming the GPT-4o-mini output as gold standard). As you noticed, the prompt in step (1) asked separately whether the document includes step-by-step instructions, and whether these are useful for training reasoning models. This gives you a choice which of the two you pick as your target - we leave that up to you - just say which you choose, i.e. do you train on those documents flagged as containing step-by-step instructions, or do you train on those documents flagged as suitable for deeper reasoning. The latter seems almost entirely a subset of the former, showing that the GPT-4o-mini model was quite smart about this task.

Further, take a sample of the data, inspect it, and judge the quality of the predictions made by your classifier. Report your findings. Further, see if you notice anything unusual about the distribution of the document genres and topics. You should be able quite easily to see how we got this data. What consequences do you think this has for your evaluation results? Are they entirely valid? And if not, why not?

Based on your evaluation, would you think it reasonable to run your classifier on the whole of FineWeb?


*   Which target did you choose?
*   Label distribution and majority baseline
*   Classifier performance
*   Manual inspection of the classifier output, what kinds of mistakes it makes?
*   What is the composition of the data we gave you? What does it mean for your results?
*   Concusions




In [105]:
#work here

MODEL_NAME = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:

from collections import defaultdict

class LogSavingCallback(transformers.TrainerCallback):
    def on_train_begin(self, *args, **kwargs):
        self.logs = defaultdict(list)
        self.training = True

    def on_train_end(self, *args, **kwargs):
        self.training = False

    def on_log(self, args, state, control, logs, model=None, **kwargs):
        if self.training:
            for k, v in logs.items():
                if k != "epoch" or v not in self.logs[k]:
                    self.logs[k].append(v)

training_logs = LogSavingCallback()

In [106]:

import evaluate
accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = outputs.argmax(axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)


data_collator = transformers.DataCollatorWithPadding(tokenizer)

# Argument gives the number of steps of patience before early stopping
early_stopping = transformers.EarlyStoppingCallback(
    early_stopping_patience=5
)



In [107]:
# Define a simple function that applies the tokenizer

# maximum length of BERT models is 512 due to the position embeddings

def tokenize(example):
    return tokenizer(
        example["document"],
        max_length=512,
        truncation=True,
    )

# Apply the tokenizer to the whole dataset using .map()
dataset = dataset.map(tokenize)



Map:   0%|          | 0/1608 [00:00<?, ? examples/s]

Map:   0%|          | 0/403 [00:00<?, ? examples/s]

In [122]:

# Set training arguments
trainer_args = transformers.TrainingArguments(
    "checkpoints",
    eval_strategy="steps",
    logging_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10,
    logging_steps=10,
    learning_rate=0.000005,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="none",
    num_train_epochs = 5
)



In [123]:
trainer = None
trainer = transformers.Trainer(
    model=model,
    args=trainer_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer = tokenizer,
    callbacks=[early_stopping, training_logs]
)

trainer.train()

<ipython-input-123-df66652555dd>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Step,Training Loss,Validation Loss,Accuracy
10,0.056900,0.220405,0.928040
20,0.112700,0.208996,0.935484
30,0.128600,0.233701,0.915633
40,0.158700,0.211552,0.928040
50,0.150100,0.205711,0.933002
60,0.104400,0.238263,0.920596
70,0.094800,0.206306,0.937965
80,0.071100,0.205524,0.933002
90,0.082500,0.202709,0.930521
100,0.059400,0.209871,0.930521


Could not locate the best model at checkpoints/checkpoint-90/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=140, training_loss=0.08898545524903706, metrics={'train_runtime': 255.5261, 'train_samples_per_second': 31.464, 'train_steps_per_second': 0.998, 'total_flos': 1166108197355520.0, 'train_loss': 0.08898545524903706, 'epoch': 2.7450980392156863})

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot(logs, keys, labels):
    values = sum([logs[k] for k in keys], [])
    plt.ylim(max(min(values)-0.1, 0.0), min(max(values)+0.1, 1.0))
    for key, label in zip(keys, labels):
        plt.plot(logs["epoch"], logs[key], label=label)
    plt.legend()
    plt.show()

plot(training_logs.logs, ["loss", "eval_loss"], ["Training loss", "Evaluation loss"])

In [ ]:
plot(training_logs.logs, ["eval_accuracy"], ["Evaluation accuracy"])

# Bonus step


(leave empty if you do not do this)

*   Prompt design
*   Build (prompt,response pairs)
*   Turn into HF Dataset and save


In [99]:
#work here

<h1> Summary and Conclusions </h1>

* Brief TL;DR -style summary and main conclusions of your project.